## Домашнее задание по блоку 19.04.2021. Кейс-стади 
### Тихонова Инна

In [57]:
import pandas as pd
import numpy as np
from scipy import stats as st
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

In [2]:
import gensim

**Задание 1**

Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

1. Как критики относятся к спортивным играм?
2. Критикам нравятся больше игры на PC или на PS4?
3. Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:
 - сформулируйте нулевую и альтернативную гипотезы;
 - выберите пороговый уровень статистической значимости;
 - опишите полученные результаты статистического теста.

In [3]:
df = pd.read_csv('video_games_sales.csv')
df.columns = df.columns.str.lower()
df.head()

,rank,name,basename,genre,esrb_rating,platform,publisher,developer,vgchartz_score,critic_score,...,na_sales,pal_sales,jp_sales,other_sales,year,last_update,url,status,vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


**1. Как критики относятся к спортивным играм?**

Критики ставят играм в основом высокие оценки (выше 7)?

 - H0: средняя оценка игр критиками <= 7.0.
 - H1: средняя оценка игр критиками > 7.0.

За alpha здесь и далее будем принимать 0.05

In [4]:
df_cs = df[np.logical_not(df['critic_score'].isna())]['critic_score']

In [5]:
alpha = 0.05
result = st.ttest_1samp(df_cs, 7.0, alternative='greater')

print(df_cs.mean())
print(result)

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, средняя оценка игр критиками выше 7.0')
else:
    print('Не отвергаем нулевую нулевую гипотезу, средняя оценка игр критиками не выше 7.0')

7.21370869033052
Ttest_1sampResult(statistic=11.882016908955192, pvalue=1.5730299883407045e-32)
Отвергаем нулевую гипотезу, средняя оценка игр критиками выше 7.0


**2. Критикам нравятся больше игры на PC или на PS4?**

Средняя оценка игр на PC и на PS4 одинакова?

 - H0: средняя оценка игр на PC и на PS4 одинакова.
 - H1: средняя оценка игр на PC и на PS4 различается.

In [6]:
df_cs_PC = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['platform'] == 'PC')]['critic_score']

df_cs_PS4 = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['platform'] == 'PS4')]['critic_score']

In [7]:
result = st.ttest_ind(df_cs_PC, df_cs_PS4, equal_var=False)
print(result)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 различается')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 одинакова')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 различается


**3. Критикам больше нравятся стрелялки или стратегии?**

Средняя оценка шутеров и стратегий одинакова?

 - H0: средняя оценка шутеров и стратегий одинакова.
 - H1: средняя оценка шутеров и стратегий различается.

In [8]:
df_cs_sh = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['genre'] == 'Shooter')]['critic_score']

df_cs_st = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['genre'] == 'Strategy')]['critic_score']

In [9]:
result = st.ttest_ind(df_cs_sh, df_cs_st, equal_var=False)
print(result)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий различается')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий одинакова')

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий одинакова


**Задание 2**

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

1. Приведите весь текст к нижнему регистру;
2. Удалите мусорные символы;
3. Удалите стоп-слова;
4. Приведите все слова к нормальной форме;
5. Преобразуйте все сообщения в вектора TF-IDF; 
--

6. Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;
7. Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;
8. Опишите результаты при помощи confusion_matrix;
9. Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [30]:
df = pd.read_csv('spam.csv')
df.columns = df.columns.str.lower()
df.head()

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


**1. Приведите весь текст к нижнему регистру**

In [43]:
text = df['message'].sum()
text[0:400]

"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...Ok lar... Joking wif u oni...Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18'sU dun say so early hor... U c already then say...Nah I don't think he goes to usf, he lives around here t"

In [44]:
text = text.lower()
text[0:400]

"go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...ok lar... joking wif u oni...free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18'su dun say so early hor... u c already then say...nah i don't think he goes to usf, he lives around here t"

**2. Удалите мусорные символы**

In [45]:
text = re.sub('[\W_]+',' ', text)
text[0:400]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat ok lar joking wif u oni free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question std txt rate t c s apply 08452810075over18 su dun say so early hor u c already then say nah i don t think he goes to usf he lives around here thoughfreemsg hey the'

**3. Удалите стоп-слова**

In [46]:
text = text.split()
text[0:10]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'n']

In [48]:
stopwords_set = set(stopwords.words('english'))
text = [word for word in text if word not in stopwords_set]
text[0:10]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la']

**4. Приведите все слова к нормальной форме**

In [49]:
wordnet_lemmatizer = WordNetLemmatizer()
text = [wordnet_lemmatizer.lemmatize(word) for word in text]
text[0:10]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la']

**5. Преобразуйте все сообщения в вектора TF-IDF**

In [51]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(text)

names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

In [53]:
tfidf_matrix.tail()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zindgi,zoe,zogtorius,zoom,zouk,zyada,ènugh,ú1,üshall,〨ud
52296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**6. Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42.**

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['category'], test_size=0.25, random_state=42)

In [58]:
lda = LinearDiscriminantAnalysis()

In [59]:
lda.fit(X_train, y_train)

ValueError: could not convert string to float: 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'